In [33]:
import pandas as pd
import numpy as np
import sklearn.metrics as skmetrics


In [2]:
loans = pd.read_csv('data/lending-club-data.csv')

/Users/sram/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'bad_loans', 'emp_length_num', 'grade_num', '

In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x==0 else -1)

In [5]:
loans = loans.drop('bad_loans',axis=1)

In [6]:
target = 'safe_loans'
features = ['grade', 'term','home_ownership','emp_length']
loans = loans[features + [target]]

In [7]:
loans.columns.values

array(['grade', 'term', 'home_ownership', 'emp_length', 'safe_loans'], dtype=object)

In [8]:
#
# one Hot encoding. 
#
loans_ohe = pd.get_dummies(loans)
loans_ohe.columns.values
features = loans_ohe.columns.values
#remove the target
features= features[1:]
#ohe_features = ohe_features[1:]
print (len(features))
print (features)
print (type(features))

25
['grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
<class 'numpy.ndarray'>


In [9]:
print (features)
x = np.where(features=='grade_A')

features = np.delete(features,x,0)

print ("AFTER====", features)


['grade_A' 'grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']
AFTER==== ['grade_B' 'grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G'
 'term_ 36 months' 'term_ 60 months' 'home_ownership_MORTGAGE'
 'home_ownership_OTHER' 'home_ownership_OWN' 'home_ownership_RENT'
 'emp_length_1 year' 'emp_length_10+ years' 'emp_length_2 years'
 'emp_length_3 years' 'emp_length_4 years' 'emp_length_5 years'
 'emp_length_6 years' 'emp_length_7 years' 'emp_length_8 years'
 'emp_length_9 years' 'emp_length_< 1 year' 'emp_length_n/a']


In [10]:
#
# load idx files for test and train data
#
import json

with open('data/module-8-assignment-2-test-idx.json','r') as f:
    test_idx = json.load(f)
f.close()
    
with open ('data/module-8-assignment-2-train-idx.json','r') as f:
    train_idx = json.load(f)
f.close()

train_data=loans_ohe.iloc[train_idx]
test_data = loans_ohe.iloc[test_idx]



        

In [11]:
def intermediate_node_weighted_mistakes(labels_in_node, data_weights):
    # Sum the weights of all entries with label +1
    #positive_nodes = [labels_in_node == +1]
    #print (positive_nodes)
    #data_weights['positive'] = positive_nodes
    
    #print (data_weights['positive'])
    #total_weight_positive = sum(data_weights[data_weights['positive']==1])
    
    #total_weight_positive = data_weights[labels_in_node.values ==1]
    #print ("mistakes: columns", data_weights.columns.values)
    #print ("mistakes: data weights type", type(data_weights))
    #print ("mistakes: labels in node", type(labels_in_node))
    #print ("*******************")
    #total_weight_positive = sum(data_weights[labels_in_node == +1])
    #print ("Data Wiehgt type {0}, shape {1}", (type(data_weights), data_weights.shape))

    total_weight_positive = pd.DataFrame.sum(data_weights[labels_in_node.values == +1])[0]
    # Sum the weights of all entries with label -1
    total_weight_negative = pd.DataFrame.sum(data_weights[labels_in_node.values == -1])[0]
    
    #print ("data_weights ", len(data_weights[labels_in_node.values == +1]))
    # Weight of mistakes for predicting all -1's is equal to the sum above
    ### YOUR CODE HERE
    # weighted_mistakes_all_negative = total_weight_positive
    # Weight of mistakes for predicting all +1's is equal to the sum above
    ### YOUR CODE HERE
    #weighted_mistakes_all_positive = total_weight_negative
    
    #print ("positve, negative", total_weight_positive, total_weight_negative)
    # Return the tuple (weight, class_label) representing the lower of the two weights
    #    class_label should be an integer of value +1 or -1.
    # If the two weights are identical, return (weighted_mistakes_all_positive,+1)
    #print ("mistake: positive, negative", (weighted_mistakes_all_positive, weighted_mistakes_all_negative))
    return (total_weight_positive, +1) if (total_weight_positive < total_weight_negative) else  (total_weight_negative,-1)

** if alpha =1, then the classification error is the same as unboosted decision stumps**

Function to pick best feature to split on

11. We continue modifying our decision tree code from the earlier assignment to incorporate weighting of individual data points. The next step is to pick the best feature to split on.

The best_splitting_feature function is similar to the one from the earlier assignment with two minor modifications:

The function best_splitting_feature should now accept an extra parameter data_weights to take account of weights of data points.
Instead of computing the number of mistakes in the left and right side of the split, we compute the weight of mistakes for both sides, add up the two weights, and divide it by the total weight of the data.

In [12]:
# If the data is identical in each feature, this function should return None

def best_splitting_feature(data,features,target, data_weights):
    target_values = data[target]
    best_feature = None
    best_error = float('+inf')
    
    #convert to float
    total_weight = pd.DataFrame.sum(data_weights,axis=0)[0]
    #print ("total weight:", total_weight)
    #print (type(total_weight))
    #num_data_points = float(len(data))
    #mfeatures = np.delete(features,0)
    #consider each feature to decide what to split on
    for feature in features:
        
        left_split = data[data[feature] == 0]        
        right_split = data[data[feature] == 1]
        
         # Apply the same filtering to data_weights to create left_data_weights, right_data_weights
        ## YOUR CODE HERE
        left_data_weights = data_weights[data[feature].values == 0]
        right_data_weights = data_weights[data[feature].values== 1]
        #left side misclassicifed
        
        left_weighted_mistakes,left_class = intermediate_node_weighted_mistakes(left_split,left_data_weights)
        right_weighted_mistakes,right_class = intermediate_node_weighted_mistakes(right_split,right_data_weights)
        
        error = float(left_weighted_mistakes + right_weighted_mistakes)/float(total_weight)
        #print ("left {0} right{1} total {2}".format(left_weighted_mistakes, right_weighted_mistakes, total_weight))
        #print ("error {0} feature {1}".format(error, feature))
        if error < best_error:
            print ("setting best feature to {0} for error {1}".format(feature, error))
            best_error = error
            best_feature = feature
        
    print ("Best Feature is %s", best_feature)
    return best_feature

In [13]:
def create_leaf(target_values, data_weights):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'is_leaf': True}
    
    # Computed weight of mistakes.
    # Store the predicted class (1 or -1) in leaf['prediction']
    weighted_error, best_class = intermediate_node_weighted_mistakes(target_values, data_weights)
    leaf['prediction'] = best_class
    
    return leaf

In [14]:
def weighted_decision_tree_create(data, features, target, data_weights, current_depth = 1, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth, data points).", (current_depth, len(target_values)))
    print ("weighted: type of data weights", type(data_weights))
    print ("weighted: len of data_weights", len(data_weights))
    print ("weighted: len of targets", len(target_values))
    
    # Stopping condition 1. Error is 0.
    if intermediate_node_weighted_mistakes(target_values, data_weights)[0] <= 1e-15:
        print ("Stopping condition 1 reached.")               
        return create_leaf(target_values, data_weights)
    
    # Stopping condition 2. No more features.
    if remaining_features == []:
        print ("Stopping condition 2 reached.")               
        return create_leaf(target_values, data_weights)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth > max_depth:
        print ("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values, data_weights)
    
    # If all the datapoints are the same, splitting_feature will be None. Create a leaf
    splitting_feature = best_splitting_feature(data, features, target, data_weights)
    #remaining_features.delete(splitting_feature,axis=1)
    splitting_feature_index = np.where(remaining_features== splitting_feature)
    #print ("REMOVAL INDEX IS", splitting_feature_index[0][0])
    #print (x[0][0])
    # delete takes the index and the axis
    remaining_features = np.delete(remaining_features,splitting_feature_index,0)
    print (remaining_features)
    #x = np.where(features=='grade_A')
    #features = np.delete(features,x,0)
    #print (remaining_features)
    #np.delete(remaining_features,splitting_feature,0)
    left_split = data[data[splitting_feature].values == 0]
    right_split = data[data[splitting_feature].values == 1]

    left_data_weights = data_weights[data[splitting_feature].values == 0]
    right_data_weights = data_weights[data[splitting_feature].values == 1]
    #print ("Left data weight ", type(left_data_weights))
    
    #print ("Split on feature ",  (\
              #splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target], data_weights)
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split[target], data_weights)
    
    # Repeat (recurse) on left and right subtrees
    left_tree = weighted_decision_tree_create(
        left_split, remaining_features, target, left_data_weights, current_depth + 1, max_depth)
    right_tree = weighted_decision_tree_create(
        right_split, remaining_features, target, right_data_weights, current_depth + 1, max_depth)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [15]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [16]:
def classify(tree, x, annotate = False):   
    # If the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print ("At leaf, predicting ", tree['prediction'])
        return tree['prediction'] 
    else:
        # Split on feature.
        #print (tree['splitting_feature'])
        #print (x)
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on ", (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [17]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    print ("evaluation: data length is ", len(data))
    print("evaluation: target len", len(data[target]))
    
    # Once you've made the predictions, calculate the classification error
    return (prediction[prediction.values != data[target].values]).sum()/float(len(data))

In [18]:
# Assign weights
example_data_weights = pd.DataFrame([1.]*10 +[0.]*(len(train_data) -20) + [1.]*10)


#print (example_data_weights)
#sum = pd.DataFrame.sum(example_data_weights, axis=0)
#print (sum[0])
#graphlab.SArray([1.] * 10 + [0.]*(len(train_data) - 20) + [1.] * 10)
# Train a weighted decision tree model.
small_data_decision_tree_subset_20 = weighted_decision_tree_create(train_data, features, target,
                         example_data_weights, max_depth=2)
print (small_data_decision_tree_subset_20)
#train_data[output.values==1]
#print (small_data_decision_tree_subset_20)

--------------------------------------------------------------------
Subtree, depth, data points). (1, 37224)
weighted: type of data weights <class 'pandas.core.frame.DataFrame'>
weighted: len of data_weights 37224
weighted: len of targets 37224
setting best feature to grade_B for error 0.5


/Users/sram/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Best Feature is %s grade_B
['grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G' 'term_ 36 months'
 'term_ 60 months' 'home_ownership_MORTGAGE' 'home_ownership_OTHER'
 'home_ownership_OWN' 'home_ownership_RENT' 'emp_length_1 year'
 'emp_length_10+ years' 'emp_length_2 years' 'emp_length_3 years'
 'emp_length_4 years' 'emp_length_5 years' 'emp_length_6 years'
 'emp_length_7 years' 'emp_length_8 years' 'emp_length_9 years'
 'emp_length_< 1 year' 'emp_length_n/a']
--------------------------------------------------------------------
Subtree, depth, data points). (2, 26858)
weighted: type of data weights <class 'pandas.core.frame.DataFrame'>
weighted: len of data_weights 26858
weighted: len of targets 26858
setting best feature to grade_C for error 0.35714285714285715
Best Feature is %s grade_C
['grade_D' 'grade_E' 'grade_F' 'grade_G' 'term_ 36 months'
 'term_ 60 months' 'home_ownership_MORTGAGE' 'home_ownership_OTHER'
 'home_ownership_OWN' 'home_ownership_RENT' 'emp_length_1 year'
 'emp_lengt

In [19]:
#train_data.head()
#small_data_decision_tree_subset_20['splitting_feature']
#train_data.iloc[:,0]
#train_data[0][small_data_decision_tree_subset_20['splitting_feature']]
#train_data.apply(lambda x: print (x))
evaluate_classification_error(small_data_decision_tree_subset_20,train_data)
#train_data['grade_A'].apply(lambda x: print (x) )
#x = small_data_decision_tree_subset_20['splitting_feature']
#print (x)
#print (type(small_data_decision_tree_subset_20))
#print (small_data_decision_tree_subset_20)
#for k in small_data_decision_tree_subset_20.keys():
#print (small_data_decision_tree_subset_20.values(k))


evaluation: data length is  37224
evaluation: target len 37224


-0.225177304964539

** Quiz Question:
Will you get the same model as small_data_decision_tree_subset_20 
if you trained a decision tree with only the 20 data points with non-zero weights from the set of points in subset_20?
**
    

No

In [20]:
def compute_adjustment(elem, weight):
    return exp(-weight) if elem == 1 else exp(weight)
compute_adjustment = np.vectorize(compute_adjustment)



In [21]:
from math import log
from math import exp

def adaboost_with_tree_stumps(data, features, target, num_tree_stumps):
    # start with unweighted data
    alpha = pd.DataFrame([1.]*len(data))
    weights = []
    tree_stumps = []
    target_values = data[target]
    
    for t in range(num_tree_stumps):
        print ('=====================================================')
        print ('Adaboost Iteration %d' % t)
        print ('=====================================================')     
        # Learn a weighted decision tree stump. Use max_depth=1
        tree_stump = weighted_decision_tree_create(data, features, target, data_weights=alpha, max_depth=1)
        tree_stumps.append(tree_stump)
        
        print ("adaboost: tree stumps", len(tree_stumps))
        
        # Make predictions
        predictions = data.apply(lambda x: classify(tree_stump, x, annotate=False), axis=1)
        
        print ("adaboost data len ", len(data))
        
        # Produce a Boolean array indicating whether
        # each data point was correctly classified
        print ("adaboost: target ", len(target_values))
        print ("adaboost: predictions ",len(predictions))
        is_correct = predictions.values == target_values.values
        is_wrong   = predictions.values != target_values.values
        print ("adaboost :correct ", is_correct)
        print ("adaboost iswrong ", is_wrong)
        print ("adaboost type of iscorrect", type(is_correct))
        
        # Compute weighted error
        # YOUR CODE HERE
        print (type(is_wrong))
        weighted_error = float(is_wrong.sum())/float((is_wrong.sum()+is_correct.sum()))
        
        # Compute model coefficient using weighted error
        # YOUR CODE HERE
        weight = 1./2.*log((1.-weighted_error)/weighted_error)

        weights.append(weight)
        #adjustment = np.apply_along_axis(is_correct.apply(lambda is_correct : exp(-weight) if is_correct else exp(weight))
        
        adjustment = compute_adjustment(is_correct, weight)
        # Scale alpha by multiplying by adjustment
        # Then normalize data points weights
        ## YOUR CODE HERE 
        print ((alpha.shape))
        print ((adjustment.shape))
        alpha = alpha[0]*adjustment
        
        alpha_sum = alpha.sum()
        alpha.apply(lambda x: x/alpha_sum)
        alpha = pd.Series.to_frame(alpha)
        print ("final type:", type(alpha))
    
    return weights, tree_stumps

In [22]:
alpha = pd.DataFrame([1.]*len(train_data))


print (alpha[0].shape)
ensemble_10 = adaboost_with_tree_stumps(train_data,features, target, num_tree_stumps=10)

(37224,)
Adaboost Iteration 0
--------------------------------------------------------------------
Subtree, depth, data points). (1, 37224)
weighted: type of data weights <class 'pandas.core.frame.DataFrame'>
weighted: len of data_weights 37224
weighted: len of targets 37224
setting best feature to grade_B for error 0.4963464431549538


/Users/sram/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Best Feature is %s grade_B
['grade_C' 'grade_D' 'grade_E' 'grade_F' 'grade_G' 'term_ 36 months'
 'term_ 60 months' 'home_ownership_MORTGAGE' 'home_ownership_OTHER'
 'home_ownership_OWN' 'home_ownership_RENT' 'emp_length_1 year'
 'emp_length_10+ years' 'emp_length_2 years' 'emp_length_3 years'
 'emp_length_4 years' 'emp_length_5 years' 'emp_length_6 years'
 'emp_length_7 years' 'emp_length_8 years' 'emp_length_9 years'
 'emp_length_< 1 year' 'emp_length_n/a']
--------------------------------------------------------------------
Subtree, depth, data points). (2, 26858)
weighted: type of data weights <class 'pandas.core.frame.DataFrame'>
weighted: len of data_weights 26858
weighted: len of targets 26858
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth, data points). (2, 10366)
weighted: type of data weights <class 'pandas.core.frame.DataFrame'>
weighted: len of data_weights 10366
weighted: len of targets 10366
Reac

In [23]:
#print (ensemble_10[1])

[{'left': {'is_leaf': True, 'splitting_feature': None, 'prediction': 1}, 'is_leaf': False, 'splitting_feature': 'grade_B', 'right': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'prediction': None}, {'left': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'is_leaf': False, 'splitting_feature': 'home_ownership_OTHER', 'right': {'is_leaf': True, 'splitting_feature': None, 'prediction': 1}, 'prediction': None}, {'left': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'is_leaf': False, 'splitting_feature': 'grade_B', 'right': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'prediction': None}, {'left': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'is_leaf': False, 'splitting_feature': 'grade_B', 'right': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'prediction': None}, {'left': {'is_leaf': True, 'splitting_feature': None, 'prediction': -1}, 'is_leaf': False, 'splitting_feature': 'home_ow

In [54]:
def predict_adaboost(stump_weights, tree_stumps, data):
    scores = pd.DataFrame([0.]*len(data))
    
    for i, tree_stump in enumerate(tree_stumps):
        predictions = data.apply(lambda x: classify(tree_stump, x),axis=1)
        
        # Accumulate predictions on scaores array
        # YOUR CODE HERE
        #print (predictions)
        print ("{0}, {1}", (len(predictions), len(scores)))
        scores = scores + predictions
        
    return scores.apply(lambda score : +1 if score.sum() > 0 else -1)

In [56]:
score = predict_adaboost(ensemble_10[0],ensemble_10[1],train_data)
print  (len(score))


{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225


In [57]:
print (len(score))
#
# don't know why score (last prediction) has one additional entry..
#
score1 = score[:-1]
#print (ensemble_10[1])
print (len(score1))



37225
37224


In [59]:
error_all = []
stump_weights = ensemble_10[0]
tree_stumps = ensemble_10[1]
for n in range(1, 31):
    predictions = predict_adaboost(stump_weights[:n], tree_stumps[:n], train_data)
    print (len(predictions))
    if (len(predictions) > 37224):
        predictions = predictions[:-1]
    error = 1.0 - skmetrics.accuracy_score(train_data[target], predictions)
    error_all.append(error)
    print ("Iteration {0}, training error = {1}", (n, error_all[n-1]))

{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (1, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (2, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (3, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (4, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (5, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
37225
Iteration {0}, training error = {1} (6, 0.5036535568450462)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0}, {1} (37224, 37224)
{0},